# Calculating magnetic declination

## Summary

This notebook contains attempts to create the Earth's magnetic field model and calculate the magnetic declination.

The first method was to use the law of cosines. Expand the law of cosines to the triangle on a sphere, and use it to calculate the magnetic declination. The triangle is created from three points, the true north, the magnetic north, and an observer. Move the observer around the earth and see how the magnetic declination varies. Draw contour line on a world map to visualize declination.

The second method was to think of the Earth's magnetic field as created from the two magnetic monopoles. Locate each monopole under the magnetic north pole and the magnetic south pole. Then calculate the magnetic field on a given point on the sphere. Using vector operations, calculate the horizontal component of the magnetic field and magnetic declination. Draw contour line and compare it with method 1.

Finally, compare both results with the WMM model, and analyze the results and errors.

## Method 1: using the law of cosines.

### Expanding the law of consines

Assume a sphere with the radius of 1. A, B, and C are three different points on the sphere and a, b, and c are their opposite sides accordingly.
Then, the following equation holds for all triangle ABC. ( $\angle$ signs are omitted for simplicity.)

$$
\cos a = \sin b \sin c \cos A + \cos b \cos c \\[0.5em]
or, \\[0.5em]
\cos A = {\cos a - \cos b \cos c \over \sin b \sin c}
$$

Thus, if we know the lenghts of three sides of a triangle on a sphere, we can then calculate angles with the formula.

### Calculating magnetic declination

Given three points, the true north, the magnetic north, and the observer, we can think of a triangle on a sphere with those points as vertices. Assuming that the magnetic declination is the same as the angle between two sides of a triangle shareing the observer point, we can calculate the magnetic declination with the above formula. 

Let's say that the true north is point A, the magnetic north is point B, and the observer is point C. Then we can get three side lengths.

$$
A = | \lambda_m - \lambda_o | \\[0.5em]
b = {\pi \over 2} - \phi_o \\[0.5em]
c = {\pi \over 2} - \phi_m \\[0.5em]
\cos a = \sin b \sin c \cos A + \cos b \cos c \\[0.7em]
\sin a = \sqrt{1 - \cos^2 a} \\[0.5em]
\cos C = {\cos c - \cos a \cos b \over \sin a \sin b}
$$

where $\phi_m$ and $\lambda_m$ are latitude and longitude of the magnetic north, and $\phi_o$ and $\lambda_o$ are those of the observer.

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Magnetic pole location was from WMM pole locations (year 2023)
# https://www.ngdc.noaa.gov/geomag/data/poles/WMM2020_NP.xy
magnetic_north_lon = 146.826
magnetic_north_lat = 86.146
magnetic_south_lon = 135.417
magnetic_south_lat = -63.930

feature_x = np.arange(-180, 180, 0.2)
feature_y = np.arange(-90, 90, 0.2)

# Observer longitude and latitude
lon, lat = np.meshgrid(feature_x, feature_y)

sign = np.sign((lon - magnetic_north_lon) % 360 - 180)
A = np.deg2rad(np.abs(magnetic_north_lon - lon))
b = np.deg2rad(90 - lat)
c = np.deg2rad(90 - magnetic_north_lat)
cos_a = np.sin(b) * np.sin(c) * np.cos(A) + np.cos(b) * np.cos(c)
sin_a = np.sqrt(1 - np.square(cos_a))
cos_C = np.clip((np.cos(c) - cos_a * np.cos(b)) / (sin_a * np.sin(b)), -1, 1)

# Magnetic declination
Z = np.rad2deg(np.arccos(cos_C)) * sign

In [ ]:
fig = go.Figure(
    data=[
        go.Contour(
            x=feature_x,
            y=feature_y,
            z=Z,
            contours=dict(
                coloring="heatmap",
                showlabels=True,  # show labels on contours
                labelfont=dict(  # label font properties
                    size=12,
                    color="white",
                ),
            ),
            showlegend=False,
            ncontours=20,
        ),
    ]
)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()
# fig.write_image("fig1.png", scale=2)

<!-- Resized, cropped for print
<div style="width: auto; height:260px; overflow:hidden;"><img src="./fig1.png" style="margin-top: -65px;height: 375px;" height="300px"></div>
-->

<img src="./fig1.png" width="75%">

### Discussion

Earth magnetic field is a complex model. In this (over) simplified model, we are missing too many things, and therefore not getting the suitable result. 
Followings are the potential reasons for the error:

 - Magnetic declination is actually based on the magnetic field's horizontal component, not the shortest path from the observer to the magnetic pole. 
 - We are not considering the magnetic south pole. This matters because the Earth's magnetic field is not symmetrical in reality.
 - The Earth is not a perfect sphere.
 

## Method 2: using vector operations

### Calculating magnetic declination

Assume the Earth's magnetic field is created from two magnetic monopoles, the south pole located under the north magnetic pole, and the north pole located under the south magnetic pole. Note that the Earth's magnetic poles and the poles of the Earth's magnetic field are the opposite.

Suppose that the Earth is a sphere with the radius of 1.
Let $\overrightarrow{p_n}$ be the position vector of the south pole of the magnetic field (or the magnetic north pole), and $\overrightarrow{p_s}$ be the position vector of the north pole of the magnetic field.

Let $\overrightarrow{x}$ be the position vector of a point on the sphere.
The relative magnetic field at $\overrightarrow{x}$ is the following

$$
\overrightarrow{B} = 
      { \overrightarrow{p_n} - \overrightarrow{x} \over 
\left\| \overrightarrow{p_n} - \overrightarrow{x} \right\| ^3 } - 
      { \overrightarrow{p_s} - \overrightarrow{x} \over 
\left\| \overrightarrow{p_s} - \overrightarrow{x} \right\| ^3 }
$$

The relative value is sufficient to calculate the direction of the magnetic field.

Note that the surface normal vector at $ \overrightarrow{x} $ is $\overrightarrow{x}$ itself.
The horizontal component of the magnetic field is the projection to the tangent plane at $\overrightarrow{x}$. To calculate it define $k$ such that

$$
( \overrightarrow{B} + k \overrightarrow{x} ) \cdot \overrightarrow{x} = 0
$$

Then the horizontal factor is
$$
\overrightarrow{d_m} = \overrightarrow{B} + k \overrightarrow{x}
$$

We can simplify the equation to calculate $k$. Since $\overrightarrow{x} \cdot \overrightarrow{x} = 1$,

$$
\overrightarrow{B} \cdot \overrightarrow{x}
+ k \overrightarrow{x} \cdot \overrightarrow{x} = 0
\\
k = -\overrightarrow{B} \cdot \overrightarrow{x}
$$



Let $\overrightarrow{x_t}$ be the position vector of the true north.
Then the direction to the north at $\overrightarrow{x}$ is $\overrightarrow{x_t} - \overrightarrow{x}$.
To get the horizontal factor, we can define $k'$ such that

$$
( (\overrightarrow{x_t} - \overrightarrow{x}) + k' \overrightarrow{x} ) \cdot \overrightarrow{x} = 0
\\
$$

$$
\begin{align*}
k' &= - (\overrightarrow{x_t} - \overrightarrow{x}) \cdot \overrightarrow{x} \\
   &= \; 1 - \overrightarrow{x_t} \cdot \overrightarrow{x}
\end{align*}
$$

The horizontal factor is
$$
\begin{align*}
\overrightarrow{d_t}
&= (\overrightarrow{x_t} - \overrightarrow{x}) + k' \overrightarrow{x} 
\\
&= \overrightarrow{x_t} - (\overrightarrow{x_t} \cdot \overrightarrow{x}) \overrightarrow{x}
\end{align*}
$$

Let $\theta$ be the magnetic declination at $\overrightarrow{x}$. Since $\theta$ is the angle between $ \overrightarrow{d_m} $ and $ \overrightarrow{d_t} $, and $ \overrightarrow{d_m} $ and $ \overrightarrow{d_t} $ are perpendicular to $ \overrightarrow{x} $, we get

$$
\cos \theta = 
{ \overrightarrow{d_m} \cdot
\overrightarrow{d_t}
\over \left\| \overrightarrow{d_m} \right\|
\left\|\overrightarrow{d_t} \right\|}
\\[1em]
\sin \theta = { \overrightarrow{d_m} \times
\overrightarrow{d_t}
\over \left\| \overrightarrow{d_m} \right\|
\left\|\overrightarrow{d_t} \right\|}
\cdot \overrightarrow{x}
$$

In [ ]:
def gcs_to_vec(lat, lon):
    x = np.cos(lat) * np.sin(lon)
    y = np.cos(lat) * np.cos(lon)
    z = np.sin(lat)
    return np.array([x, y, z])


pn_depth = 3600 / 6400
ps_depth = 3600 / 6400


In [ ]:
# Observer longitude and latitude
lon, lat = np.meshgrid(feature_x, feature_y)

x = gcs_to_vec(np.deg2rad(lat), np.deg2rad(lon))

pn = gcs_to_vec(np.deg2rad(magnetic_north_lat), np.deg2rad(magnetic_north_lon))
pn = pn * pn_depth
pn = np.broadcast_to(np.reshape(pn, (3, 1, 1)), np.shape(x))
ps = gcs_to_vec(np.deg2rad(magnetic_south_lat), np.deg2rad(magnetic_south_lon))
ps = ps * ps_depth
ps = np.broadcast_to(np.reshape(ps, (3, 1, 1)), np.shape(x))

x_t = np.broadcast_to(np.reshape([0, 0, 1], (3, 1, 1)), np.shape(x))

pn_min_x = pn - x
pn_min_x_len_cubed = np.power(np.sum(np.square(pn_min_x), axis=0), 1.5)

ps_min_x = ps - x
ps_min_x_len_cubed = np.power(np.sum(np.square(ps_min_x), axis=0), 1.5)

B = pn_min_x / pn_min_x_len_cubed - ps_min_x / ps_min_x_len_cubed
k = -np.sum(B * x, axis=0)
d_m = B + k * x
# Normalize d_m
d_m = d_m / np.sqrt(np.sum(np.square(d_m), axis=0))

# Since x_t is [0, 0, 1], x_t dot x is the z component of x.
d_t = x_t - x[2] * x
# Normalize d_t
d_t = d_t / np.sqrt(np.sum(np.square(d_t), axis=0))

cos_theta = np.clip(np.sum(d_t * d_m, axis=0), -1, 1)
sin_theta = np.clip(np.sum(x * np.cross(d_t, d_m, 0, 0, 0), axis=0), -1, 1)

# Magnetic declination
theta = np.rad2deg(np.arccos(cos_theta)) * np.sign(sin_theta)

In [ ]:
fig = go.Figure(
    data=[
        go.Contour(
            x=feature_x,
            y=feature_y,
            z=theta,
            contours=dict(
                coloring="heatmap",
                showlabels=True,  # show labels on contours
                labelfont=dict(  # label font properties
                    size=12,
                    color="white",
                ),
            ),
            showlegend=False,
            ncontours=20,
        ),
    ]
)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()
# fig.write_image("fig2.png", scale=2)

<!-- Resized, cropped for print
<div style="width: auto; height:260px; overflow:hidden;"><img src="./fig2.png" style="margin-top: -65px;height: 375px;" height="300px"></div>
-->

<img src="./fig2.png" width="75%">